In [1]:
import navis
import fafbseg
import pymaid

In [2]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [3]:
from pyroglancer.layers import create_nglayer
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.ngviewer import openviewer, closeviewer
from pyroglancer.ngspaces import create_ngspace

# Step 1: Fetch stuff from flywire..

In [4]:
#set the flywire token here..
#fafbseg.flywire.set_chunkedgraph_secret("blabla")

In [5]:
#mesh in flywire for which to find a match..
flywire_id = 720575940633919863

In [6]:
# Fetch the neuron's mesh
putativeDNa02_a = fafbseg.flywire.get_mesh_neuron(flywire_id)
putativeDNa02_a

,
type,navis.MeshNeuron
name,None
id,720575940633919863
units,1 nanometer
n_vertices,785994
n_faces,1575396


# Step 2: Transform it to fafb..

In [7]:
DNa02_xyz = pd.DataFrame(list(map(np.ravel, putativeDNa02_a.vertices)), columns=['x', 'y', 'z'])

In [8]:
#move the vertices of the flywire mesh to fafb..
fafb_putativeDNa02_xyz = navis.xform_brain(DNa02_xyz, source='FLYWIRE', target='FAFB14')

Transform path: FLYWIRE -> FLYWIREraw -> FAFB14raw -> FAFB14


In [9]:
fafb_putativeDNa02_a = putativeDNa02_a.copy()

In [10]:
#put the transformed vertices in the new mesh..
fafb_putativeDNa02_a.vertices = fafb_putativeDNa02_xyz.to_numpy()

In [11]:
#perform a bit of conversion to navis volumes..
fafb_putativeDNa02_a = navis.core.volumes.Volume(fafb_putativeDNa02_a)
fafb_putativeDNa02_a.id = flywire_id
fafb_putativeDNa02_a.name = 'putative_DNa02'
fafb_putativeDNa02_a

<navis.Volume(name=putative_DNa02, id=720575940633919863, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(785994, 3), faces.shape=(1575396, 3))>

# Step 3: fetch neuropils from catmaid for display..

In [12]:
#set up catmaid..
working_rm = pymaid.CatmaidInstance('https://neuropil.janelia.org/tracing/fafb/v14',
                            api_token=os.environ['TOKEN'],
                            http_user=os.environ['authname'], # omit if not required
                            http_password=os.environ['authpassword'], project_id = 1) # omit if not required

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


In [13]:
#fetch the LAL neuropil mesh..
LAL_neuropil = pymaid.get_volume('LAL_L', color=(255, 0, 0, .2))

In [14]:
#perform a bit of conversion to navis volumes..
LAL_neuropil = navis.core.volumes.Volume(LAL_neuropil)
LAL_neuropil.id = 100
LAL_neuropil.name = 'LAL_L'
LAL_neuropil

<navis.Volume(name=LAL_L, id=100, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(402, 3), faces.shape=(800, 3))>

In [15]:
#fetch the FB neuropil mesh..
FB_neuropil = pymaid.get_volume('FB', color=(0, 255, 0, .2))

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


In [16]:
#perform a bit of conversion to navis volumes..
FB_neuropil = navis.core.volumes.Volume(FB_neuropil)
FB_neuropil.id = 200
FB_neuropil.name = 'FB'
FB_neuropil

<navis.Volume(name=FB, id=200, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(614, 3), faces.shape=(1224, 3))>

# Step 4: Set up pyroglancer configuration..

In [17]:
startdataserver()

Serving data from:  /var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz
Serving directory at http://127.0.0.1:8000


In [18]:
create_ngspace('FAFB')

using default location at:  ~/.pyroglancer/config.yml
Neuroglancer viewer created at:  http://127.0.0.1:52033/v/a3926ac8162233f9d927c05db60f1f753ec2e575/
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  image
using default location at:  ~/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  image
using default location at:  ~/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  surfacemesh
using default location at:  ~/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  segmentation
using default location at:  ~/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  synapsepred
using default location

# Step 5: Inject the neuron mesh and neuropils..

In [19]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': fafb_putativeDNa02_a,
                                        'name': 'flywiremeshes','color': 'yellow'})

using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  volumes
mesh/720575940633919863
Seg id is: 720575940633919863
Full filepath: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz/precomputed/flywiremeshes/mesh/720575940633919863
creating: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz/precomputed/flywiremeshes/mesh/segment_properties
creating: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz/precomputed/flywiremeshes/mesh/segment_names
Using layout : xy-3d


127.0.0.1 - - [04/Feb/2021 21:33:09] "GET /precomputed/flywiremeshes/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:09] "GET /precomputed/flywiremeshes/mesh/segment_properties/info HTTP/1.1" 200 -


In [20]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': [LAL_neuropil,FB_neuropil],
                                        'name': 'neuropils','color': ['magenta', 'cyan'], 'alpha': 0.3})

using default location at:  ~/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  volumes
mesh/100
Seg id is: 100
Full filepath: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz/precomputed/neuropils/mesh/100
mesh/200
Seg id is: 200
Full filepath: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz/precomputed/neuropils/mesh/200
creating: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz/precomputed/neuropils/mesh/segment_properties
creating: /private/var/folders/6x/d_rd5yr937vbq8f46d4yzrsw0000gn/T/tmpg5u9mbiz/precomputed/neuropils/mesh/segment_names
Using layout : xy-3d


127.0.0.1 - - [04/Feb/2021 21:33:14] "GET /precomputed/neuropils/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:14] "GET /precomputed/neuropils/mesh/segment_properties/info HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:27] "GET /precomputed/neuropils/mesh/100:0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:27] "GET /precomputed/neuropils/mesh/100 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:28] "GET /precomputed/neuropils/mesh/200:0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:28] "GET /precomputed/neuropils/mesh/200 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:35] "GET /precomputed/flywiremeshes/mesh/720575940633919863:0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2021 21:33:35] "GET /precomputed/flywiremeshes/mesh/720575940633919863 HTTP/1.1" 200 -


# Step 6: After injection poke around the flywire mesh

### The yellow mesh is the flywire neuron mesh and the green mesh is the fafb poke around match. The magenta region is the LAL, and the cyan region is the FB. The red dots are presynapses, the blue dots are postsynapses. You can see that the putativeDNa02_a is indeed getting many synapses from LAL

![alt text](pics/flywire2fafb.png "Title")

In [ ]:
closeviewer()

In [ ]:
closedataserver()